In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets)=boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 6us/step


In [2]:
print('train_data: ', train_data.shape)
print('test_data: ', test_data.shape)

train_data:  (404, 13)
test_data:  (102, 13)


In [ ]:
train_data.shape[1]

13

In [3]:
mean=train_data.mean(axis=0)
std=train_data.std(axis=0)

train_data-=mean
train_data/=std

test_data-=mean
test_data/=std



*  입력 데이터 feature들의 범위가 서로 다르므로, 스케일을 조정해주는 전처리 단계를 해줘야 한다.


In [8]:
from keras import layers
from keras import models

def build_model():
  model=models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',
                loss='mse',
                metrics=['mae'])
  return model



*   회귀에서는 손실 함수로 MSE(평균 제곱 오차), 평가 지표로 MAE(평균 절대 오차)를 주로 사용한다.


In [5]:
import numpy as np

k=4
num_val_samples=len(train_data)//k

print(num_val_samples)

101




---


---




*   기존의 train, validation, test set

＊전체 training set을 (partial_x_train, x_val), (partial_y_train, y_val)로 나눠서 학습을 진행한다.



    history=model.fit(partial_x_train,  partial_y_train, epcohs=20, batch_size=512, validation_data=(x_val,y_val))






  acc, loss, val_acc, val_loss 에 대해 그래프를 그려보고 과대적합(Overfitting)이 일어나는 지점을 확인한다.




  만약 두번째 epoch 이후부터 training set에 과도하게 최적화 되었으면, epoch을 4까지 줄인다.




  ＊최종 모델에 대해서 학습하므로, 원래의 전체 데이터(x_train, y_train) 에 대해서 학습(fit)을 진행한다.

    model.fit(x_train, y_train, epochs=4, batch_size=512)
    test_loss, test_acc=model.evaluate(x_test,y_test)
    model.predict(x_test)




---





*   K-겹 검증에서의 train, validation, test set

＊모델의 최종 검증 점수는 k개의 검증 점수의 평균이 된다.

＊k개의 검증 점수 : 하나의 fold 에서의 'validation'은 모델의 성능을 평가하는 test set역할을 하므로 fit에서 validation_data를 사용하지 않는다.

    model.fit(partial_train_data, partial_train_targets, epochs=100, batch_size=1)
    val_mse, val_mae=model.evaluate(val_data, val_targets) 

*val_data, val_targets 가 위에서의 test set역할을 하므로 검증데이터에 대해서 'evaluate'를 한다.

*모델 성능 평가를 위한 loss funtion으로 'mse', 성능 측정 매트릭스로 'mae'를 사용했다.


---



---



In [9]:
num_epochs=100
all_scores=[]

for i in range(k):
  print('처리중인 폴드 #',i)
  val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]

  partial_train_data = np.concatenate([train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)   # np.concatenate: numpy 배열을 합쳐준다.
  partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)

  model=build_model()
  model.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs,
            batch_size=1,
            verbose=0)    # verbose=0 이므로 훈련 과정이 출력되지 않는다.
  val_mse, val_mae=model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)  # 검증 점수의 평균을 사용 할 것이기 때문에, 성능 측정 매트릭스의 점수를 저장한다.

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [11]:
np.mean(all_scores)

2.434341609477997



*   (최종 모델의 검증 점수는) K개의 검증 점수의 평균이다.



In [12]:
model=build_model()
model.fit(train_data, train_targets,
          epochs=80,              # 'MAE'그래프를 그려보면, 80번째 epoch 이후부터 과대적합이 시작되는 것을 확인할 수 있다.
          batch_size=16,
          verbose=0)

test_mse_score, test_mae_score=model.evaluate(test_data, test_targets)

print(test_mae_score)

102/102 [==============================] - 0s 176us/step
2.52256441116333
